# Sentiment Analysis


## 1. Librerías

In [9]:
import pickle
import pandas as pd
import os
import requests
import re
import string
import sys
import meaningcloud
from collections import Counter 

## 2. Función sentiment y primera prueba
Primero, definimos la función analizadora sentiment e intentamos utilizarla en uno de los textos del corpus

In [ ]:
#Información importante
license_key = "" #se puede obtener registrándose en la página de meaningcloud.com
diccionario = {'ilang': 'es'} #se puede definir otros parámetros
#diccionario = 'migraciones'


#Definición de la función para hacer el análisis de sentimiento
def sentiment(texto):
    sentiment_response = meaningcloud.SentimentResponse(meaningcloud.SentimentRequest(license_key, lang='es', txt=texto, txtf='plain', otherparams=diccionario).sendReq())
    return(sentiment_response.getResponse())

#definición de función para quitar saltos de línea
def clean(text):
    '''Get rid of some additional punctuation and non-sensical text that was missed the first time around.'''
    text = re.sub('\n', ' ', text)
    return(text)

In [16]:
#abrimos el corpus y cargamos tan solo una nota para la prueba
with open("Corpus/El video sobre la brutal represión de la Policía de la Ciudad que censuró Facebook - ELDESTAPE", encoding='utf-8') as file:
    texto = file.read()

In [17]:
#Hacemos el análisis de sentimiento de esa nota e imprimimos el diccionario
sentiment(clean(texto))

{'status': {'code': '0',
  'msg': 'OK',
  'credits': '2',
  'remaining_credits': '18661'},
 'model': 'general_es',
 'score_tag': 'N',
 'agreement': 'DISAGREEMENT',
 'subjectivity': 'SUBJECTIVE',
 'confidence': '86',
 'irony': 'NONIRONIC',
 'sentence_list': [{'text': 'El video sobre la brutal represión de la Policía de la Ciudad que censuró Facebook Se trata de una grabación en la que se ve cómo los efectivos golpean a un vendedor senegalés con una fractura expuesta en el brazo.',
   'inip': '0',
   'endp': '214',
   'bop': 'y',
   'confidence': '100',
   'score_tag': 'N',
   'agreement': 'AGREEMENT',
   'segment_list': [{'text': 'El video sobre la brutal represión de la Policía de la Ciudad que censuró Facebook Se trata de una grabación en la que se ve cómo los efectivos golpean a un vendedor senegalés con una fractura expuesta en el brazo',
     'segment_type': 'main',
     'inip': '0',
     'endp': '213',
     'confidence': '100',
     'score_tag': 'N',
     'agreement': 'AGREEMENT',

## 3. Análisis de sentimiento de las noticias del corpus
Ahora, cargamos todo nuestro corpus y lo analizamos con sentiment

In [18]:
corpus = pickle.load(open("corpus.pkl", "rb")) #sacamos el corpus de su pickle (df de 87 x 3)

In [19]:
corpus_noticias = corpus.iloc[:,2] #seleccionamos tan solo las notas (columna 2)

"\n#código para analizar la primera noticia del corpus\nnoticia = corpus_noticias[0]\ntexto = texto.encode('utf-8')\nsentimento = sentiment(noticia)\n"

In [34]:
#A continuación, analizamos todas las noticias del corpus
sentiment_analysis = []
for noticia in corpus_noticias:
    analisis_sent = sentiment(clean(noticia))
    sentiment_analysis.append(analisis_sent)

In [35]:
ej = sentiment_analysis[78]
ej

{'status': {'code': '0',
  'msg': 'OK',
  'credits': '1',
  'remaining_credits': '18384'},
 'model': 'general_es',
 'score_tag': 'N',
 'agreement': 'DISAGREEMENT',
 'subjectivity': 'SUBJECTIVE',
 'confidence': '86',
 'irony': 'NONIRONIC',
 'sentence_list': [{'text': 'Prohibido ser negro en Buenos Aires 05/06/2018 Por Cosecha Roja 0 Un grupo de Infantería de la Policía de la Ciudad empuja a Kane Serigne Dame, un ciudadano Senegalés, contra una vidriera.',
   'inip': '1',
   'endp': '189',
   'bop': 'y',
   'confidence': '100',
   'score_tag': 'N',
   'agreement': 'AGREEMENT',
   'segment_list': [{'text': 'Prohibido ser negro en Buenos Aires 05/06/2018 Por Cosecha Roja 0 Un grupo de Infantería de la Policía de la Ciudad',
     'segment_type': 'secondary',
     'inip': '1',
     'endp': '116',
     'confidence': '100',
     'score_tag': 'NONE',
     'agreement': 'AGREEMENT',
     'polarity_term_list': [],
     'sentimented_entity_list': [{'form': 'Buenos Aires',
       'id': '5d2df9543aa4

In [36]:
#guardamos nuestra lista con pickle
pickle.dump(sentiment_analysis, open("corpus_sentimiento", "wb"))

## 4. Organización de los datos en un panda df

In [37]:
#Abrimos nuestro corpus
sentiment_analysis = pickle.load(open("corpus_sentimiento", "rb"))
corpus = pickle.load(open("corpus.pkl", "rb"))
analisis_sent = sentiment_analysis

In [74]:
#Código para representar la info del análisis en un df
score_tag = []
agreement = []
subjectivity = []
confidence = []
irony = []

for d in analisis_sent:
    score_tag.append(d['score_tag'])
    agreement.append(d['agreement'])
    subjectivity.append(d['subjectivity'])
    confidence.append(int(d['confidence']))
    irony.append(d['irony'])
    
ZippedList = list(zip(score_tag, agreement, subjectivity, irony, confidence)) #Convertimos nuestras cinco listas en una lista de tuplas
df =pd.DataFrame(ZippedList, columns = ['Polaridad', 'Acuerdo', 'Subjetividad','Ironía', 'Confianza'])


In [75]:
df

,Polaridad,Acuerdo,Subjetividad,Ironía,Confianza
0,N,DISAGREEMENT,SUBJECTIVE,NONIRONIC,86
1,N,DISAGREEMENT,SUBJECTIVE,NONIRONIC,86
2,N,DISAGREEMENT,SUBJECTIVE,IRONIC,76
3,N,DISAGREEMENT,SUBJECTIVE,NONIRONIC,86
4,NEU,DISAGREEMENT,SUBJECTIVE,IRONIC,76
5,N,DISAGREEMENT,SUBJECTIVE,IRONIC,76
6,NEU,DISAGREEMENT,SUBJECTIVE,NONIRONIC,86
7,NEU,DISAGREEMENT,SUBJECTIVE,NONIRONIC,91
8,P,DISAGREEMENT,SUBJECTIVE,IRONIC,76
9,NEU,DISAGREEMENT,SUBJECTIVE,NONIRONIC,82


In [45]:
#prueba de un diccionario
sentiment_analysis[59]

{'status': {'code': '0',
  'msg': 'OK',
  'credits': '4',
  'remaining_credits': '18410'},
 'model': 'general_es',
 'score_tag': 'NEU',
 'agreement': 'DISAGREEMENT',
 'subjectivity': 'SUBJECTIVE',
 'confidence': '76',
 'irony': 'IRONIC',
 'sentence_list': [{'text': 'Inmigrantes senegaleses son víctimas de violencia en Argentina Por Daniel Gutman © Reproducir este artículo| | Imprimir | |English version La violencia de la Policía de Buenos Aires hacia vendedores ambulantes senegaleses en los últimos meses han indignado a argentinos e inmigrantes que el 16 de junio se manifestaron contra estos hechos en la capital.',
   'inip': '1',
   'endp': '353',
   'bop': 'y',
   'confidence': '100',
   'score_tag': 'N+',
   'agreement': 'AGREEMENT',
   'segment_list': [{'text': 'Inmigrantes senegaleses son víctimas de violencia en Argentina Por Daniel Gutman © Reproducir este artículo| | Imprimir | |English version La violencia de la Policía de Buenos Aires hacia vendedores ambulantes senegaleses e

In [76]:
#concatenamos el resultado del análisis con el corpus
analisis_final_sent = pd.concat([corpus,df], axis=1)

In [77]:
analisis_final_sent

,mes,Título,Noticia,Polaridad,Acuerdo,Subjetividad,Ironía,Confianza
0,Enero,Se oficializó el decreto de necesidad y urgenc...,Se oficializó el decreto de necesidad y urgenc...,N,DISAGREEMENT,SUBJECTIVE,NONIRONIC,86
1,Enero,El desprecio - PAGINA 12,29 de enero de 2017\nEl desprecio\nAunque pare...,N,DISAGREEMENT,SUBJECTIVE,NONIRONIC,86
2,Enero,Preocupación y desconcierto” por la reforma mi...,25 de enero de 2017\nMás de 130 organizaciones...,N,DISAGREEMENT,SUBJECTIVE,IRONIC,76
3,Enero,El Gobierno modificó la Ley de Migraciones con...,30 de enero de 2017\nEl Gobierno modificó la L...,N,DISAGREEMENT,SUBJECTIVE,NONIRONIC,86
4,Enero,Con el DNU todos los migrantes son sospechosos...,Críticas del CELS y Amnistía Internacional a l...,NEU,DISAGREEMENT,SUBJECTIVE,IRONIC,76
5,Febrero,"El éxodo de venezolanos, una preocupación para...","El éxodo de venezolanos, una preocupación para...",N,DISAGREEMENT,SUBJECTIVE,IRONIC,76
6,Febrero,Bolivia rechazó el pedido de la Argentina para...,Bolivia rechazó el pedido de la Argentina para...,NEU,DISAGREEMENT,SUBJECTIVE,NONIRONIC,86
7,Febrero,El 70% de los argentinos cree que los inmigran...,El 70% de los argentinos cree que los inmigran...,NEU,DISAGREEMENT,SUBJECTIVE,NONIRONIC,91
8,Febrero,Marcos Peña se manifestó a favor de que el Con...,Marcos Peña se manifestó a favor de que el Con...,P,DISAGREEMENT,SUBJECTIVE,IRONIC,76
9,Febrero,La lupa sobre los inmigrantes: ¿qué impacto re...,La lupa sobre los inmigrantes: ¿qué impacto re...,NEU,DISAGREEMENT,SUBJECTIVE,NONIRONIC,82


In [78]:
pickle.dump(analisis_final_sent, open("sentiment_final", "wb")) #guardo nuestro nuevo dataframe en un pickle

## 5. Análisis estadístico de los datos

In [79]:
analisis_final_sent = pickle.load(open("sentiment_final", "rb"))
#analisis_final_sent.to_csv('analisis de sentimiento.csv', encoding='utf-8', index=False, sep=',')

In [68]:
#vemos esadísticas básicas de nuestro dataframe, que nos tira el análisis de confidence
analisis_final_sent.describe()

,Confianza
count,87.000000
mean,85.781609
std,4.542884
min,74.000000
25%,84.000000
50%,86.000000
75%,89.000000
max,94.000000


In [80]:
#especificamos un df con solo las columnas de score_tag, agreement, subjectivity y irony
analisis_simple = analisis_final_sent.iloc[:, 3:-1]

In [81]:
analisis_simple

,Polaridad,Acuerdo,Subjetividad,Ironía
0,N,DISAGREEMENT,SUBJECTIVE,NONIRONIC
1,N,DISAGREEMENT,SUBJECTIVE,NONIRONIC
2,N,DISAGREEMENT,SUBJECTIVE,IRONIC
3,N,DISAGREEMENT,SUBJECTIVE,NONIRONIC
4,NEU,DISAGREEMENT,SUBJECTIVE,IRONIC
5,N,DISAGREEMENT,SUBJECTIVE,IRONIC
6,NEU,DISAGREEMENT,SUBJECTIVE,NONIRONIC
7,NEU,DISAGREEMENT,SUBJECTIVE,NONIRONIC
8,P,DISAGREEMENT,SUBJECTIVE,IRONIC
9,NEU,DISAGREEMENT,SUBJECTIVE,NONIRONIC


In [53]:
#Código de prueba para saber cuántos valores únicos (niveles) tiene una variable
analisis_simple.iloc[:,0].nunique()

3


1. Analizamos las polaridades (polaridad) de manera global 

In [56]:
cuenta_score_tags = analisis_simple['Polaridad'].value_counts(normalize=True) * 100

In [57]:
cuenta_score_tags 

N      58.620690
P      24.137931
NEU    17.241379
Name: Polaridad, dtype: float64

2. Analizamos las polaridades (agreement) de manera global 

In [58]:
cuenta_agreement = analisis_simple['Acuerdo'].value_counts(normalize=True) * 100

In [59]:
cuenta_agreement

DISAGREEMENT    94.252874
AGREEMENT        5.747126
Name: Acuerdo, dtype: float64

3. Analizamos las polaridades (subjectividad) de manera global

In [82]:
cuenta_subjectivity = analisis_simple['Subjetividad'].value_counts(normalize=True) * 100
cuenta_subjectivity

SUBJECTIVE    97.701149
OBJECTIVE      2.298851
Name: Subjetividad, dtype: float64

4. Analizamos las polaridades (Ironía) de manera global y mensual

In [83]:
cuenta_irony = analisis_simple['Ironía'].value_counts(normalize=True) * 100
cuenta_irony

NONIRONIC    87.356322
IRONIC       12.643678
Name: Ironía, dtype: float64